In [ ]:
import pandas as pd

In [ ]:
# Load weekly summary
df = pd.read_csv("macro_weekly_summary.csv")

In [ ]:
# Step 1: Remove rows with zero users
df = df[df['users_covered'] > 0]

In [ ]:
# Step 2: Aggregate total margin per week
margin_df = df.groupby('week').agg({'Margin': 'sum'}).reset_index()

In [ ]:
# Step 3: Prepare for Prophet
df_margin_prophet = margin_df.rename(columns={'week': 'ds', 'Margin': 'y'})
df_margin_prophet['ds'] = pd.to_datetime(df_margin_prophet['ds'].str.split('/').str[0])

In [ ]:
# Step 4: Prepare for LSTM
df_margin_lstm = df_margin_prophet.copy()

In [ ]:
# Preview
print("✅ Prophet-ready Margin data:")
print(df_margin_prophet)

✅ Prophet-ready Margin data:
          ds             y
0 2020-12-28  21086.337693
1 2021-01-04  52421.210479
2 2021-01-11  60402.464113
3 2021-01-18  57320.067696
4 2021-01-25  54720.064012
5 2021-02-01  55028.101104
6 2021-02-08  50125.361322
7 2021-02-15  52375.708858
8 2021-02-22  55116.434773
9 2021-03-01   4498.903223


In [ ]:
print("\n✅ LSTM-ready Margin series:")
print(df_margin_lstm['y'].values)


✅ LSTM-ready Margin series:
[21086.33769308 52421.21047894 60402.46411336 57320.06769558
 54720.06401201 55028.10110423 50125.36132216 52375.70885769
 55116.43477332  4498.90322283]


In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from prophet import Prophet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
### Step 1: Prepare data
train_prophet = df_margin_lstm.iloc[:8].copy()
test_prophet = df_margin_lstm.iloc[8:].copy()
train_prophet.columns = ['ds', 'y']
test_prophet.columns = ['ds', 'y']

In [ ]:
# Prophet model
prophet_model = Prophet()
prophet_model.fit(train_prophet)
future = prophet_model.make_future_dataframe(periods=2, freq='W-MON')
forecast = prophet_model.predict(future)
pred_prophet = forecast[['ds', 'yhat']].iloc[-2:]['yhat'].values
actual_margin = test_prophet['y'].values

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 5.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp517r2s19/ell821ew.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp517r2s19/kmg74gar.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=92785', 'data', 'file=/tmp/tmp517r2s19/ell821ew.json', 'init=/tmp/tmp517r2s19/kmg74gar.json', 'output', 'file=/tmp/tmp517r2s19/prophet_modelu8l9wwlt/prophet_model-20250620182303.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
18:23:03 - cmdstanpy - INFO - Chain [1] start processing
IN

In [ ]:
# LSTM setup
margin_series = df_margin_lstm['y'].values.reshape(-1, 1)
scaler = MinMaxScaler()
scaled_margin = scaler.fit_transform(margin_series)

In [ ]:
# Prepare sequences
X, y = [], []
seq_len = 4
for i in range(0, 8 - seq_len):
    X.append(scaled_margin[i:i + seq_len])
    y.append(scaled_margin[i + seq_len])
X = np.array(X)
y = np.array(y)
X = X.reshape((X.shape[0], X.shape[1], 1))

In [ ]:
# Train LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(seq_len, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Forecast next 2 weeks
last_seq = scaled_margin[8 - seq_len:8].reshape(1, seq_len, 1)
lstm_preds = []
for _ in range(2):
    pred = model.predict(last_seq)[0][0]
    lstm_preds.append(pred)
    last_seq = np.append(last_seq[:, 1:, :], [[[pred]]], axis=1)
pred_lstm = scaler.inverse_transform(np.array(lstm_preds).reshape(-1, 1)).flatten()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [ ]:
### Step 2: Compare & Evaluate

print("📊 Actual Margin:", actual_margin)
print("📈 Prophet Forecast:", pred_prophet)
print("🧠 LSTM Forecast:", pred_lstm)

📊 Actual Margin: [55116.43477332  4498.90322283]
📈 Prophet Forecast: [60546.5411704  62795.76497529]
🧠 LSTM Forecast: [51992.383 51411.68 ]


In [ ]:
# Step 3: Evaluation
mae_prophet = mean_absolute_error(actual_margin, pred_prophet)
rmse_prophet = np.sqrt(mean_squared_error(actual_margin, pred_prophet))

mae_lstm = mean_absolute_error(actual_margin, pred_lstm)
rmse_lstm = np.sqrt(mean_squared_error(actual_margin, pred_lstm))

print(f"\n✅ Prophet → MAE: {mae_prophet:.2f}, RMSE: {rmse_prophet:.2f}")
print(f"✅ LSTM    → MAE: {mae_lstm:.2f}, RMSE: {rmse_lstm:.2f}")


✅ Prophet → MAE: 31863.48, RMSE: 41400.54
✅ LSTM    → MAE: 25018.41, RMSE: 33245.81
